<a href="https://colab.research.google.com/github/Hiszat/Data-Sains/blob/main/Pertemuan_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
tweets = pd.read_csv("/content/drive/MyDrive/DataSains/BaksoSudahDiProses.csv")

In [ ]:
!pip install tensorflow
!pip install Keras-Preprocessing==1.1.2
!pip install scikeras
!pip install keras==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')

!pip install twint
import twint
import nest_asyncio
nest_asyncio.apply()

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.5/477.5 kB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.2 MB/s eta 0:00:00
  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33912 sha256=deef9ab06c9dd64d114712834f0b78b3b2fd85e2965d82cf17a04655abe338f7
  Stored in directory: /root/.cache/pip/wheels/48/d2/f5/7a71ea8b6788b85fac82abf4ef84dac03562571f695c3b8f77
  Created wheel for cchardet: filename=cchardet-2.1.7-cp310-cp310-linux_x86_64.whl size=289391 sha256=fb68a1f70a1e7970826a3f57bac893fc4101323030cbf4d7d0700f3675658df1
  Stored in directory: /root/.cache/pip/wheel

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import  Embedding, Dense, Dropout, LSTM
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text)
  text = re.sub(r'#[A-Za-z0-9]+', '', text)
  text = re.sub(r'RT[\s]', '', text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'[0-9]+', '', text)

  text = text.replace('\n', ' ')
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = text.strip(' ')
  return text

def casefoldingText(text):
  text = text.lower()
  return text

def tokenizingText(text):
  text = word_tokenize(text)
  return text

def filteringText(text):
  listStopwords = set(stopwords.words('indonesian'))
  filtered = []
  for txt in text:
    if txt not in listStopwords:
      filtered.append(txt)

  text = filtered
  return text

def stemmingText(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  text = [stemmer.stem(word) for word in text]
  return text

def toSentence(list_words):
  sentence = ' '.join(word for word in list_words)
  return sentence

In [ ]:
X = tweets['tweet_baku'].apply(toSentence)
max_features = 5000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X.values)
X = tokenizer.texts_to_sequences(X.values)
X = pad_sequences(X)
X.shape

(605, 345)

In [ ]:
from sklearn.model_selection import train_test_split

polarity_encode = {'negative': 0, 'neutral': 1, 'positive': 2}
y = tweets['polarity'].map(polarity_encode).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(484, 345) (484,)
(121, 345) (121,)


In [ ]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense

def create_model(embed_dim=16,hidden_unit=16, dropout_rate=0.2, optimizer=Adam, learning_rate=0.001):
  model = Sequential()
  model.add(Embedding(input_dim=max_features, output_dim=embed_dim, input_length=X_train.shape[1]))
  model.add(LSTM(units=hidden_unit, activation='tanh'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(units=3, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer(lr=learning_rate), metrics=['accuracy'])
  print(model.summary())
  return model


In [ ]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense

# Fungsi untuk membuat model
def create_model(embed_dim=16,hidden_unit=16, optimizer=Adam, learning_rate=0.001):
  model = Sequential()
  model.add(Embedding(input_dim=max_features, output_dim=embed_dim, input_length=X_train.shape[1]))
  model.add(LSTM(units=hidden_unit, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(Dense(units=3, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer(lr=learning_rate), metrics=['accuracy'])
  print(model.summary())
  return model

# Hyperparameters
param_grid = {
    'embed_dim': [32, 64],
    'hidden_unit': [16, 32, 64],
    'optimizer': ['Adam', 'RMSprop'],
    'learning_rate': [0.1, 0.01, 0.001, 0.0001],
    'epochs': [10, 25, 50, 100],
    'batch_size': [128, 256]
}

# Wrapper model with default hyperparameter values
model = KerasClassifier(build_fn=create_model, embed_dim=32, hidden_unit=16, learning_rate=0.01, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

In [ ]:
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 345, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 16)                3136      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 3)                 51        
                                                                 
Total params: 163187 (637.45 KB)
Trainable params: 163187 (637.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


None


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 345, 32)           160000    
                                                                 
 lstm_1 (LSTM)               (None, 16)                3136      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 51        
                                                                 
Total params: 163187 (637.45 KB)
Trainable params: 163187 (637.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


...

Model: "sequential_1150"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1150 (Embedding)  (None, 345, 64)           320000    
                                                                 
 lstm_1150 (LSTM)            (None, 64)                33024     
                                                                 
 dropout_1150 (Dropout)      (None, 64)                0         
                                                                 
 dense_1150 (Dense)          (None, 3)                 195       
                                                                 
Total params: 353219 (1.35 MB)
Trainable params: 353219 (1.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


None


Model: "sequential_1151"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1151 (Embedding)  (None, 345, 64)           320000    
                                                                 
 lstm_1151 (LSTM)            (None, 64)                33024     
                                                                 
 dropout_1151 (Dropout)      (None, 64)                0         
                                                                 
 dense_1151 (Dense)          (None, 3)                 195       
                                                                 
Total params: 353219 (1.35 MB)
Trainable params: 353219 (1.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


ValueError: 
All the 1152 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1152 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 928, in _fit
    self._ensure_compiled_model()
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 439, in _ensure_comp

In [ ]:
results = pd.DataFrame(grid_result.cv_results_)
results.to_csv('gridsearchcv_results.csv', index=False)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
results = pd.read_csv()
results.sort_values(by='mean_score_time', ascending = False).reset_index(drop = True)